In [1]:
import pandas as pd
import os
import json

In [2]:
def read_json(ConfigPath):
    with open(ConfigPath,'r') as f:
        return json.load(f)

In [3]:
IntermediatePath = r"C:\sunil\Tasks\sai\stdf versions\7March\STDF\STDF\Data\Intermediate/"
ConfigPath = r"C:\sunil\Tasks\sai\stdf versions\7March\STDF\STDF\Config/"
filename = "a595"

In [4]:
Mar_Percentage=read_json(ConfigPath+'configpath.json').get("Mar_Percentage")
masterfile = IntermediatePath+filename+'.xlsx'    
ptr_df_columns = ['TEST_NUM','LO_LIMIT', 'HI_LIMIT', 'UNITS','TEST_TXT']
ftr_df_columns = ['TEST_NUM','TEST_TXT']
mir_df_columns=["PART_TYP"]
mir_df = pd.read_excel(masterfile, sheet_name = 'MIR')[mir_df_columns]
DeviceName = mir_df.PART_TYP[0]
ptr_presence=True    
ftr_presence=True    

In [5]:
try:
    ptr_df = pd.read_excel(masterfile, sheet_name = 'PTR')[ptr_df_columns]
    ptr_unique=ptr_df.drop_duplicates(subset="TEST_NUM")
    ptr_unique["T_Type"]="PTR"
except:
    print("Sheet Not available")
    ptr_presence=False
try:
    ftr_df = pd.read_excel(masterfile, sheet_name = 'FTR')[ftr_df_columns]
    ftr_unique=ftr_df.drop_duplicates(subset="TEST_NUM")
    ftr_unique["T_Type"]="FTR"
except:
    print("Sheet Not available")
    ftr_presence=False    
if ftr_presence and ptr_presence:
    df=pd.concat([ptr_unique,ftr_unique])
elif ptr_presence:
    df=ptr_unique    
elif ftr_presence:
    df=ftr_unique    
else:
    print("No sheet available")    

<ipython-input-5-9d84ee1398e5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptr_unique["T_Type"]="PTR"
<ipython-input-5-9d84ee1398e5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ftr_unique["T_Type"]="FTR"


In [11]:
df["Percentage"]=Mar_Percentage/100
df["MAR_HI_UP_LIMIT"]=df["HI_LIMIT"]+df["Percentage"]*(df["HI_LIMIT"])
df["MAR_HI_DOWN_LIMIT"]=df["HI_LIMIT"]-df["Percentage"]*(df["HI_LIMIT"])
df["MAR_LO_UP_LIMIT"]=df["LO_LIMIT"]+df["Percentage"]*(df["LO_LIMIT"])
df["MAR_LO_DOWN_LIMIT"]=df["LO_LIMIT"]-df["Percentage"]*(df["LO_LIMIT"])
if len(df.loc[df['LO_LIMIT']==0]) > 0:
    ddf = pd.DataFrame()
    ddf = df.loc[df['LO_LIMIT']==0]
    ddf["MAR_LO_UP_LIMIT"]=ddf["LO_LIMIT"]+ddf["Percentage"]*(ddf["HI_LIMIT"])
    ddf["MAR_LO_DOWN_LIMIT"]=ddf["LO_LIMIT"]-ddf["Percentage"]*(ddf["HI_LIMIT"])
    for index,row in ddf.iterrows():
        df.loc[df['TEST_NUM'] == row['TEST_NUM'], 'MAR_LO_UP_LIMIT'] = row['MAR_LO_UP_LIMIT']
        df.loc[df['TEST_NUM'] == row['TEST_NUM'], 'MAR_LO_DOWN_LIMIT'] = row['MAR_LO_DOWN_LIMIT']

<ipython-input-11-9678d31ff2fa>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddf["MAR_LO_UP_LIMIT"]=ddf["LO_LIMIT"]+ddf["Percentage"]*(ddf["HI_LIMIT"])
<ipython-input-11-9678d31ff2fa>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddf["MAR_LO_DOWN_LIMIT"]=ddf["LO_LIMIT"]-ddf["Percentage"]*(ddf["HI_LIMIT"])


In [ ]:
for index,row in dff.iterrows():
    df.loc[df['TEST_NUM'] == row['TEST_NUM'], 'MAR_LO_UP_LIMIT'] = row['MAR_LO_UP_LIMIT']
    df.loc[df['TEST_NUM'] == row['TEST_NUM'], 'MAR_LO_DOWN_LIMIT'] = row['MAR_LO_DOWN_LIMIT']

In [12]:
df

Device  TEST_NUM      LO_LIMIT      HI_LIMIT UNITS  \
0      mcc       100           NaN           NaN   NaN   
1      mcc       110           NaN           NaN   NaN   
2      mcc       200  1.000000e+00  3.000000e+00     v   
3      mcc       210  3.500000e-02  5.500000e-02     a   
4      mcc       220  3.500000e-02  6.000000e-02     a   
..     ...       ...           ...           ...   ...   
118    mcc       743  3.430000e-07  3.560000e-07     s   
119    mcc       744  3.450000e-07  3.600000e-07     s   
120    mcc       745  3.480000e-07  3.630000e-07     s   
121    mcc       746  3.500000e-07  3.670000e-07     s   
122    mcc       747  3.520000e-07  3.710000e-07     s   

                         TEST_TXT T_Type  Percentage  MAR_HI_UP_LIMIT  \
0             Open/Short- <> cont    FTR        0.03              NaN   
1             Open/Short+ <> cont    FTR        0.03              NaN   
2      VDD supply <> setup_device    PTR        0.03     3.090000e+00   
3              IDD_Static <> curr    PTR        0.03     5.665000e-02   
4          IDD1 @ 1.25MHz <> curr    PTR        0.03     6.180000e-02   
..                            ...    ...         ...              ...   
118  Delay Line #44 <> delay_line    PTR        0.03     3.666800e-07   
119  Delay Line #45 <> delay_line    PTR        0.03     3.708000e-07   
120  Delay Line #46 <> delay_line    PTR        0.03     3.738900e-07   
121  Delay Line #47 <> delay_line    PTR        0.03     3.780100e-07   
122  Delay Line #48 <> delay_line    PTR        0.03     3.821300e-07   

     MAR_HI_DOWN_LIMIT  MAR_LO_UP_LIMIT  MAR_LO_DOWN_LIMIT  
0                  NaN              NaN                NaN  
1                  NaN              NaN                NaN  
2         2.910000e+00     1.030000e+00       9.700000e-01  
3         5.335000e-02     3.605000e-02       3.395000e-02  
4         5.820000e-02     3.605000e-02       3.395000e-02  
..                 ...              ...                ...  
118       3.453200e-07     3.532900e-07       3.327100e-07  
119       3.492000e-07     3.553500e-07       3.346500e-07  
120       3.521100e-07     3.584400e-07       3.375600e-07  
121       3.559900e-07     3.605000e-07       3.395000e-07  
122       3.598700e-07     3.625600e-07       3.414400e-07  

[123 rows x 12 columns]

In [14]:
#df.to_csv(ConfigPath+'TEST_NUM.csv')    
config=ConfigPath+'TEST_NUM.csv'    
if not os.path.exists(config):
    df.insert(0,"Device",DeviceName,True)
    #master_file(ConfigPath,1)
    df.sort_values(by='TEST_NUM').to_csv(ConfigPath+'TEST_NUM.csv',index=False)
df_existing = pd.read_csv(config)
#max_grp=df_existing["Group"].max()+1    
new_df = df.loc[~df.TEST_NUM.isin(df_existing.TEST_NUM)]
new_df["Device"]=DeviceName    
if len(new_df): 
    #master_file(ConfigPath,max_grp)
    df=pd.concat([df_existing,new_df]).sort_values(by='TEST_NUM',ignore_index=True)
    df.to_csv(ConfigPath+'TEST_NUM.csv',index=False)

In [ ]:
def ConfigFile(IntermediatePath,ConfigPath,filename):
    Mar_Percentage=read_json(ConfigPath+'configpath.json').get("Mar_Percentage")
    masterfile = IntermediatePath+filename+'.xlsx'    
    ptr_df_columns = ['TEST_NUM','LO_LIMIT', 'HI_LIMIT', 'UNITS','TEST_TXT']
    ftr_df_columns = ['TEST_NUM','TEST_TXT']
    mir_df_columns=["PART_TYP"]
    mir_df = pd.read_excel(masterfile, sheet_name = 'MIR')[mir_df_columns]
    DeviceName = mir_df.PART_TYP[0]
    ptr_presence=True    
    ftr_presence=True    
    try:
        ptr_df = pd.read_excel(masterfile, sheet_name = 'PTR')[ptr_df_columns]
        ptr_unique=ptr_df.drop_duplicates(subset="TEST_NUM")
        ptr_unique["T_Type"]="PTR"
    except:
        print("Sheet Not available")
        ptr_presence=False
    try:
        ftr_df = pd.read_excel(masterfile, sheet_name = 'FTR')[ftr_df_columns]
        ftr_unique=ftr_df.drop_duplicates(subset="TEST_NUM")
        ftr_unique["T_Type"]="FTR"
    except:
        print("Sheet Not available")
        ftr_presence=False    
    if ftr_presence and ptr_presence:
        df=pd.concat([ptr_unique,ftr_unique])
    elif ptr_presence:
        df=ptr_unique    
    elif ftr_presence:
        df=ftr_unique    
    else:
        return "No sheet available"    
    df["Percentage"]=Mar_Percentage/100
    df["MAR_HI_UP_LIMIT"]=df["HI_LIMIT"]+df["Percentage"]*(df["HI_LIMIT"])
    df["MAR_HI_DOWN_LIMIT"]=df["HI_LIMIT"]-df["Percentage"]*(df["HI_LIMIT"])
    df["MAR_LO_UP_LIMIT"]=df["LO_LIMIT"]+df["Percentage"]*(df["LO_LIMIT"])
    df["MAR_LO_DOWN_LIMIT"]=df["LO_LIMIT"]-df["Percentage"]*(df["LO_LIMIT"])
    if len(df.loc[df['LO_LIMIT']==0]) > 0:
        df = df.loc[df['LO_LIMIT']==0]
        df["MAR_LO_UP_LIMIT"]=df["LO_LIMIT"]+df["Percentage"]*(df["HI_LIMIT"])
        df["MAR_LO_DOWN_LIMIT"]=df["LO_LIMIT"]-df["Percentage"]*(df["HI_LIMIT"])
    #df.to_csv(ConfigPath+'TEST_NUM.csv')    
    config=ConfigPath+'TEST_NUM.csv'    
    if not os.path.exists(config):
        df.insert(0,"Device",DeviceName,True)
        #master_file(ConfigPath,1)
        df.sort_values(by='TEST_NUM').to_csv(ConfigPath+'TEST_NUM.csv',index=False)
    df_existing = pd.read_csv(config)
    #max_grp=df_existing["Group"].max()+1    
    new_df = df.loc[~df.TEST_NUM.isin(df_existing.TEST_NUM)]
    new_df["Device"]=DeviceName    
    if len(new_df): 
        #master_file(ConfigPath,max_grp)
        df=pd.concat([df_existing,new_df]).sort_values(by='TEST_NUM',ignore_index=True)
        df.to_csv(ConfigPath+'TEST_NUM.csv',index=False)

In [ ]:
import pandas as pd
import os
import json
def read_json(ConfigPath):
    with open(ConfigPath,'r') as f:
        return json.load(f)
def ConfigFile(IntermediatePath,ConfigPath,filename):
    Mar_Percentage=read_json(ConfigPath+'configpath.json').get("Mar_Percentage")
    masterfile = IntermediatePath+filename+'.xlsx'    
    ptr_df_columns = ['TEST_NUM','LO_LIMIT', 'HI_LIMIT', 'UNITS','TEST_TXT']
    ftr_df_columns = ['TEST_NUM','TEST_TXT']
    mir_df_columns=["PART_TYP"]
    mir_df = pd.read_excel(masterfile, sheet_name = 'MIR')[mir_df_columns]
    DeviceName = mir_df.PART_TYP[0]
    ptr_presence=True    
    ftr_presence=True    
    try:
        ptr_df = pd.read_excel(masterfile, sheet_name = 'PTR')[ptr_df_columns]
        ptr_unique=ptr_df.drop_duplicates(subset="TEST_NUM")
        ptr_unique["T_Type"]="PTR"
    except:
        print("Sheet Not available")
        ptr_presence=False
    try:
        ftr_df = pd.read_excel(masterfile, sheet_name = 'FTR')[ftr_df_columns]
        ftr_unique=ftr_df.drop_duplicates(subset="TEST_NUM")
        ftr_unique["T_Type"]="FTR"
    except:
        print("Sheet Not available")
        ftr_presence=False    
    if ftr_presence and ptr_presence:
        df=pd.concat([ptr_unique,ftr_unique])
    elif ptr_presence:
        df=ptr_unique    
    elif ftr_presence:
        df=ftr_unique    
    else:
        return "No sheet available"    
    df["Percentage"]=Mar_Percentage/100
    df["MAR_HI_UP_LIMIT"]=df["HI_LIMIT"]+df["Percentage"]*(df["HI_LIMIT"])
    df["MAR_HI_DOWN_LIMIT"]=df["HI_LIMIT"]-df["Percentage"]*(df["HI_LIMIT"])
    df["MAR_LO_UP_LIMIT"]=df["LO_LIMIT"]+df["Percentage"]*(df["LO_LIMIT"])
    df["MAR_LO_DOWN_LIMIT"]=df["LO_LIMIT"]-df["Percentage"]*(df["LO_LIMIT"])
    if len(df.loc[df['LO_LIMIT']==0]) > 0:
        df = df.loc[df['LO_LIMIT']==0]
        df["MAR_LO_UP_LIMIT"]=df["LO_LIMIT"]+df["Percentage"]*(df["HI_LIMIT"])
        df["MAR_LO_DOWN_LIMIT"]=df["LO_LIMIT"]-df["Percentage"]*(df["HI_LIMIT"])
    #df.to_csv(ConfigPath+'TEST_NUM.csv')    
    config=ConfigPath+'TEST_NUM.csv'    
    if not os.path.exists(config):
        df.insert(0,"Device",DeviceName,True)
        #master_file(ConfigPath,1)
        df.sort_values(by='TEST_NUM').to_csv(ConfigPath+'TEST_NUM.csv',index=False)
    df_existing = pd.read_csv(config)
    #max_grp=df_existing["Group"].max()+1    
    new_df = df.loc[~df.TEST_NUM.isin(df_existing.TEST_NUM)]
    new_df["Device"]=DeviceName    
    if len(new_df): 
        #master_file(ConfigPath,max_grp)
        df=pd.concat([df_existing,new_df]).sort_values(by='TEST_NUM',ignore_index=True)
        df.to_csv(ConfigPath+'TEST_NUM.csv',index=False)
# def master_file(ConfigPath,Group_cnt):
#     path=ConfigPath+'Master_Test_Num.csv'    
#     df=pd.DataFrame(columns=['Group_id','Group_Name'],data=[[Group_cnt,f'Group_{Group_cnt}']])
#     if os.path.exists(path):
#         exist_df=pd.read_csv(path)
#         df=exist_df.append(df)
#     df.to_csv(path,index=False)
#ConfigFile(IntermediatePath,ConfigPath)

In [7]:
def ConfigFile(IntermediatePath,ConfigPath,filename):
    Mar_Percentage=read_json(ConfigPath+'configpath.json').get("Mar_Percentage")
    masterfile = IntermediatePath+filename+'.xlsx'    
    ptr_df_columns = ['TEST_NUM','LO_LIMIT', 'HI_LIMIT', 'UNITS','TEST_TXT']
    ftr_df_columns = ['TEST_NUM','TEST_TXT']
    mir_df_columns=["PART_TYP"]
    mir_df = pd.read_excel(masterfile, sheet_name = 'MIR')[mir_df_columns]
    DeviceName = mir_df.PART_TYP[0]
    ptr_presence=True    
    ftr_presence=True    
    try:
        ptr_df = pd.read_excel(masterfile, sheet_name = 'PTR')[ptr_df_columns]
        ptr_unique=ptr_df.drop_duplicates(subset="TEST_NUM")
        ptr_unique["T_Type"]="PTR"
    except:
        print("Sheet Not available")
        ptr_presence=False
    try:
        ftr_df = pd.read_excel(masterfile, sheet_name = 'FTR')[ftr_df_columns]
        ftr_unique=ftr_df.drop_duplicates(subset="TEST_NUM")
        ftr_unique["T_Type"]="FTR"
    except:
        print("Sheet Not available")
        ftr_presence=False    
    if ftr_presence and ptr_presence:
        df=pd.concat([ptr_unique,ftr_unique])
    elif ptr_presence:
        df=ptr_unique    
    elif ftr_presence:
        df=ftr_unique    
    else:
        return "No sheet available"    
    df["Percentage"]=Mar_Percentage/100
    df["MAR_HI_UP_LIMIT"]=df["HI_LIMIT"]+df["Percentage"]*(df["HI_LIMIT"])
    df["MAR_HI_DOWN_LIMIT"]=df["HI_LIMIT"]-df["Percentage"]*(df["HI_LIMIT"])
    df["MAR_LO_UP_LIMIT"]=df["LO_LIMIT"]+df["Percentage"]*(df["LO_LIMIT"])
    df["MAR_LO_DOWN_LIMIT"]=df["LO_LIMIT"]-df["Percentage"]*(df["LO_LIMIT"])
    if len(df.loc[df['LO_LIMIT']==0]) > 0:
        ddf = pd.DataFrame()
        ddf = df.loc[df['LO_LIMIT']==0]
        ddf["MAR_LO_UP_LIMIT"]=ddf["LO_LIMIT"]+ddf["Percentage"]*(ddf["HI_LIMIT"])
        ddf["MAR_LO_DOWN_LIMIT"]=ddf["LO_LIMIT"]-ddf["Percentage"]*(ddf["HI_LIMIT"])
        for index,row in ddf.iterrows():
            df.loc[df['TEST_NUM'] == row['TEST_NUM'], 'MAR_LO_UP_LIMIT'] = row['MAR_LO_UP_LIMIT']
            df.loc[df['TEST_NUM'] == row['TEST_NUM'], 'MAR_LO_DOWN_LIMIT'] = row['MAR_LO_DOWN_LIMIT']
    #df.to_csv(ConfigPath+'TEST_NUM.csv')    
    config=ConfigPath+'TEST_NUM.csv'    
    if not os.path.exists(config):
        df.insert(0,"Device",DeviceName,True)
        #master_file(ConfigPath,1)
        if ftr_presence ==True:
            df.sort_values(by='TEST_NUM').to_csv(ConfigPath+'TEST_NUM.csv',index=False)
        else:
            df.to_csv(ConfigPath+'TEST_NUM.csv',index=False)

    df_existing = pd.read_csv(config)
    #max_grp=df_existing["Group"].max()+1    
    new_df = df.loc[~df.TEST_NUM.isin(df_existing.TEST_NUM)]
    new_df["Device"]=DeviceName    
    if len(new_df): 
        #master_file(ConfigPath,max_grp)
        
        # df=pd.concat([df_existing,new_df]).sort_values(by='TEST_NUM',ignore_index=True)
        # df.to_csv(ConfigPath+'TEST_NUM.csv',index=False)
        if ftr_presence ==True:
            df=pd.concat([df_existing,new_df]).sort_values(by='TEST_NUM',ignore_index=True)
            df.to_csv(ConfigPath+'TEST_NUM.csv',index=False)
        else:
            df=pd.concat([df_existing,new_df],ignore_index=True)
            df.to_csv(ConfigPath+'TEST_NUM.csv',index=False)

In [8]:
# IntermediatePath = r"C:\sunil\Tasks\sai\stdf versions\Base\Data\Intermediate/"
# ConfigPath = r"C:\sunil\Tasks\sai\stdf versions\Base\Config/"
# filename = "a595"

In [10]:
IntermediatePath = r"C:\sunil\Tasks\sai\stdf versions\SBH66\Data\Intermediate/"
ConfigPath = r"C:\sunil\Tasks\sai\stdf versions\Base\Config/"
filename = "STDF66"



In [11]:
ConfigFile(IntermediatePath,ConfigPath,filename)

<ipython-input-7-9a123bcc05d4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptr_unique["T_Type"]="PTR"


Sheet Not available


<ipython-input-7-9a123bcc05d4>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Percentage"]=Mar_Percentage/100
<ipython-input-7-9a123bcc05d4>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["MAR_HI_UP_LIMIT"]=df["HI_LIMIT"]+df["Percentage"]*(df["HI_LIMIT"])
<ipython-input-7-9a123bcc05d4>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [13]:
file = r"C:\sunil\Tasks\sai\stdf versions\Base\Config\TEST_NUM.csv"
df1 = pd.read_csv(file)
df1.head()

Device  TEST_NUM  LO_LIMIT  HI_LIMIT UNITS                    TEST_TXT  \
0    mcc       100       NaN       NaN   NaN         Open/Short- <> cont   
1    mcc       110       NaN       NaN   NaN         Open/Short+ <> cont   
2    mcc       200     1.000     3.000     v  VDD supply <> setup_device   
3    mcc       210     0.035     0.055     a          IDD_Static <> curr   
4    mcc       220     0.035     0.060     a      IDD1 @ 1.25MHz <> curr   

  T_Type  Percentage  MAR_HI_UP_LIMIT  MAR_HI_DOWN_LIMIT  MAR_LO_UP_LIMIT  \
0    FTR        0.03              NaN                NaN              NaN   
1    FTR        0.03              NaN                NaN              NaN   
2    PTR        0.03          3.09000            2.91000          1.03000   
3    PTR        0.03          0.05665            0.05335          0.03605   
4    PTR        0.03          0.06180            0.05820          0.03605   

   MAR_LO_DOWN_LIMIT  
0                NaN  
1                NaN  
2            0.97000  
3            0.03395  
4            0.03395

In [15]:
df1.Device.unique()[0]

'mcc'

In [12]:
data = {'Name': ['Alice', 'Bob', 'Charlie', 'Dave', 'Eve'],
        'Age': [25, 30, 35, 40, 45],
        'City': ['New York', 'Paris', 'London', 'Tokyo', 'New York']}
df = pd.DataFrame(data)

# specify the value and column to search for
search_value = 'New York'
search_column = 'City'

# filter the dataframe based on the search condition
filtered_df = df[df[search_column] == search_value]

# print the filtered dataframe
print(filtered_df)

    Name  Age      City
0  Alice   25  New York
4    Eve   45  New York


In [17]:
import os

folder_path = "C:\delete\common/"

# loop through all files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path) # delete the file
            print(f"{filename} deleted successfully.")
    except Exception as e:
        print(f"Error deleting {filename}: {e}")

common.csv deleted successfully.
common_10054.csv deleted successfully.
common_10083.csv deleted successfully.
common_101167.csv deleted successfully.
common_101395.csv deleted successfully.
common_11417.csv deleted successfully.
common_17093.csv deleted successfully.
common_1835.csv deleted successfully.
common_18762.csv deleted successfully.
common_20000.csv deleted successfully.
common_20546.csv deleted successfully.
common_21653.csv deleted successfully.
common_226.csv deleted successfully.
common_23572.csv deleted successfully.
common_26547.csv deleted successfully.
common_30915.csv deleted successfully.
common_31944.csv deleted successfully.
common_35124.csv deleted successfully.
common_43739.csv deleted successfully.
common_49278.csv deleted successfully.
common_6399.csv deleted successfully.
common_68183.csv deleted successfully.
common_74264.csv deleted successfully.
common_74312.csv deleted successfully.
common_90533.csv deleted successfully.
common_90799.csv deleted successf

In [62]:
folder_path = "C:\delete\Back\_2023-05-04_20-20-02_Thursday\Device/"
for file_or_folder in os.listdir(folder_path):
    path_to_delete = os.path.join(folder_path, file_or_folder)
    if os.path.isdir(path_to_delete):
        shutil.rmtree(path_to_delete)
    elif os.path.isfile(path_to_delete):
        os.remove(path_to_delete)
    else:
        print(f"'{file_or_folder}' does not exist or is not a file or folder.")

In [110]:

def DeleteFiles_Folders(folder_path):
    files_and_folders_to_delete = ["Device", "Input", "Intermediate","Output","Reports","Summery"]

    for fdelete in files_and_folders_to_delete:
        folder_path1 = folder_path+fdelete+'/'
        for file_or_folder in os.listdir(folder_path1):
            print(file_or_folder)
            path_to_delete = os.path.join(folder_path1, file_or_folder)
            #path_to_delete = folder_path+file_or_folder
            print(path_to_delete)
            if os.path.isdir(path_to_delete):
                shutil.rmtree(path_to_delete)
            elif os.path.isfile(path_to_delete):
                os.remove(path_to_delete)
            else:
                pass

In [111]:
folder_path = r"C:\delete\Back/_2023-05-05_10-53-56_Friday/"
DeleteFiles_Folders(folder_path)

mcc
C:\delete\Back/_2023-05-05_10-53-56_Friday/Device/mcc
a595.stdf
C:\delete\Back/_2023-05-05_10-53-56_Friday/Input/a595.stdf
a595.xlsx
C:\delete\Back/_2023-05-05_10-53-56_Friday/Intermediate/a595.xlsx
CheckingforFTR.csv
C:\delete\Back/_2023-05-05_10-53-56_Friday/Intermediate/CheckingforFTR.csv
df_final.csv
C:\delete\Back/_2023-05-05_10-53-56_Friday/Intermediate/df_final.csv
Final_output.xlsx
C:\delete\Back/_2023-05-05_10-53-56_Friday/Intermediate/Final_output.xlsx
Mapped_PRR.xlsx
C:\delete\Back/_2023-05-05_10-53-56_Friday/Intermediate/Mapped_PRR.xlsx
Record_types.xlsx
C:\delete\Back/_2023-05-05_10-53-56_Friday/Intermediate/Record_types.xlsx
TransposedData.csv
C:\delete\Back/_2023-05-05_10-53-56_Friday/Intermediate/TransposedData.csv
a595.xlsx
C:\delete\Back/_2023-05-05_10-53-56_Friday/Output/a595.xlsx
HBIN.csv
C:\delete\Back/_2023-05-05_10-53-56_Friday/Output/HBIN.csv
OverallHbin.jpg
C:\delete\Back/_2023-05-05_10-53-56_Friday/Output/OverallHbin.jpg
OverallHbin.xlsx
C:\delete\Back/_20

In [5]:
import os
#folder_path = "C:\delete\Back/_2023-05-05_10-35-45_Friday/Device/"
folder_path = "C:\sunil\Tasks\sai\stdf versions\Base\Data\Input"
#folder_path = folder_path.replace(r'\','')
for file_or_folder in os.listdir(folder_path):
    path_to_delete = os.path.join(folder_path, file_or_folder)
    print(path_to_delete)
    if os.path.isdir(path_to_delete):
        shutil.rmtree(path_to_delete)
        print(f"Folder '{file_or_folder}' deleted successfully.")
    elif os.path.isfile(path_to_delete):
        os.remove(path_to_delete)
        #print(f"File '{file_or_folder}' deleted successfully.")
    else:
        print(f"'{file_or_folder}' does not exist or is not a file or folder.")

C:\sunil\Tasks\sai\stdf versions\Base\Data\Input\a595.stdf
C:\sunil\Tasks\sai\stdf versions\Base\Data\Input\SBH36427609_1544580967100.stdf
C:\sunil\Tasks\sai\stdf versions\Base\Data\Input\STDF66.std


In [108]:
def Backup_Data(Path,BackupPath):
    current_time = datetime.datetime.now().strftime("_%Y-%m-%d_%H-%M-%S_%A")
    folder_name = f"Backup_{current_time}"
    source_path = Path
    destination_path = BackupPath+current_time+'/'

    files_and_folders_to_copy = ["Device", "Input", "Intermediate","Output","Reports","Summery"]

    for file_or_folder in files_and_folders_to_copy:
        source = os.path.join(source_path, file_or_folder)
        destination = os.path.join(destination_path, file_or_folder)
        try:
            if os.path.isdir(source):
                shutil.copytree(source, destination)
                #print(f"Folder '{file_or_folder}' copied successfully.")
            elif os.path.isfile(source):
                shutil.copy2(source, destination)
                #print(f"File '{file_or_folder}' copied successfully.")
            else:
                print(f"'{file_or_folder}' does not exist or is not a file or folder.")
        except Exception as e:
            print(f"Error while Backup the {filename}: {e}")
    print(destination_path)

In [109]:
Path = 'C:\delete/'
BackupPath = 'C:\delete\Back/'

Backup_Data(Path,BackupPath)

C:\delete\Back/_2023-05-05_10-53-56_Friday/


In [22]:
import shutil

folder_path = "C:\delete\ewfolder/" 
if os.path.exists(folder_path) is True:
    shutil.rmtree(folder_path)

In [ ]:
def DelFolders(FoldersList):
    for f in FoldersList:
        if os.path.exists(folder_path) is False:
            shutil.rmtree(folder_path)
def DeleteFiles_in_Folder(FoldersList):
    for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path) # delete the file
            #print(f"{filename} deleted successfully.")
    except Exception as e:
        print(f"Error deleting {filename}: {e}")

In [ ]:
import os,shutil
import pathlib

In [ ]:
toolP = str(pathlib.Path(pathlib.Path(__file__).parent.absolute()))
ToolPath = toolP+'/..'
print('ToolPath : ',ToolPath)
json_obj = json_reader(ToolPath+'/Config/configpath.json')

InputPath = ToolPath+json_obj['InputPath']
IntermediatePath = ToolPath+json_obj['IntermediatePath']
DevicePath = ToolPath+json_obj['DevicePath']
SummeryPath = ToolPath+json_obj['SummeryPath']
OutputPath = ToolPath+json_obj['OutputPath']
# BackupPath = /Data/

BackupPath = ToolPath+json_obj['BackupPath']
DltEntire = [DevicePath]
DltOnlyFiles = [InputPath,IntermediatePath,SummeryPath,OutputPath]



In [ ]:
source_path = "path/to/source/folder"
destination_path = "path/to/destination/folder"
folders_to_copy = ["Device", "Input", "Intermediate","Output","Reports","Summery"]

for folder in folders_to_copy:
    source_folder = os.path.join(source_path, folder)
    destination_folder = os.path.join(destination_path, folder)
    shutil.copytree(source_folder, destination_folder)

In [ ]:
import shutil
import os

source_path = "path/to/source/folder"
destination_path = "path/to/destination/folder"
files_and_folders_to_copy = ["Device", "Input", "Intermediate","Output","Reports","Summery"]

for file_or_folder in files_and_folders_to_copy:
    source = os.path.join(source_path, file_or_folder)
    destination = os.path.join(destination_path, file_or_folder)
    if os.path.isdir(source):
        shutil.copytree(source, destination)
        #print(f"Folder '{file_or_folder}' copied successfully.")
    elif os.path.isfile(source):
        shutil.copy2(source, destination)
        #print(f"File '{file_or_folder}' copied successfully.")
    else:
        print(f"'{file_or_folder}' does not exist or is not a file or folder.")

In [28]:
import datetime
current_time = datetime.datetime.now().strftime("%H:%M:%S")
print("Current time is:", current_time)

Current time is: 17:46:37


In [30]:
current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
print("Current date and time is:", current_time)

Current date and time is: 2023-05-04_17:48:49


In [ ]:
os.makedirs(current_time)

In [ ]:
current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S_%A")
folder_name = f"Backup_{current_time}"

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created successfully.")
else:
    print(f"Folder '{folder_name}' already exists.")

In [31]:
current_time = datetime.datetime.now()
date_str = current_time.strftime("%Y-%m-%d")
time_str = current_time.strftime("%H:%M:%S")
day_str = current_time.strftime("%A")

print("Current date and time is:", date_str, time_str)
print("Current day is:", day_str)

Current date and time is: 2023-05-04 17:50:38
Current day is: Thursday


In [32]:
datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S_%A")

'2023-05-04_17-51-12_Thursday'

In [1]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return 'Hello, world!'

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/May/2023 23:27:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2023 23:27:09] "GET /favicon.ico HTTP/1.1" 404 -
